In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
songs = pd.read_csv('songdata.csv')

In [4]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
songs = songs.sample(n=20000).drop('link', axis=1).reset_index(drop=True)

In [6]:
songs['text'] = songs['text'].str.replace(r'\n', '')

<ipython-input-6-215fe46be90d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [7]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [8]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [9]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [10]:
similarities = {}

In [11]:
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [22]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)
recommedations = ContentBasedRecommender(similarities)

In [23]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}
recommedations.recommend(recommendation)

The 4 recommended songs for Only A Miracle are:
Number 1:
My Oh My It's A Miracle by Gino Vannelli with 0.57 similarity score
--------------------
Number 2:
Some Kind Of Miracle by Kelly Clarkson with 0.483 similarity score
--------------------
Number 3:
One Small Miracle by Bryan White with 0.407 similarity score
--------------------
Number 4:
I Need A Miracle by Grateful Dead with 0.365 similarity score
--------------------


In [33]:
#User input
songip=input("Enter song title: ")
recommendation1 = {
    "song": songip,
    "number_songs": 4
}
recommedations.recommend(recommendation1)

Enter song title: Let Go
The 4 recommended songs for Let Go are:
Number 1:
Millenium by Outkast with 0.564 similarity score
--------------------
Number 2:
Dynamite by Christina Aguilera with 0.513 similarity score
--------------------
Number 3:
Alone Together by Fall Out Boy with 0.509 similarity score
--------------------
Number 4:
Sometimes by Lady Gaga with 0.502 similarity score
--------------------
